In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
# from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
# import numpy as np
# import sklearn
import pickle
from keras.applications import *
from keras.preprocessing.image import *
# importing dependencies
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import tqdm ##
from tqdm import * ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import xgboost as xgb
import lightgbm as lgb
import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Take Label

In [2]:
label = pd.read_csv('data/train_label.csv',names=['ID','fname','label','verified'],header=0)
dicts_ = pickle.load(open('data/map.pkl','rb'))
label['trans']=label['label'].map(dicts_)
# label = label.drop(['ID','fname','verified','label'],axis=1)
label = label['trans'].values
print('label like:',label,'data#:', len(label))

label like: [13 34  7 ... 37 39  7] data#: 1895


## Take Training Data

In [32]:
folder = 'data/10fold_stacking/' #共同predict對unverified data的結果

acc_df = pd.read_csv('data/ens_unverified/validation_ACC_P1S1.csv') #accuracy csv
acc_df.columns = ['model','csv_name','acc','other']
acc_df = acc_df.filter(['csv_name','acc'])
files = os.listdir(folder)

# ratio_all=0
a=0
b=0
c=0
d=0
for i,csv in enumerate(files):
    if csv.startswith('validation_ACC'):
        continue
    elif csv.startswith('mike_resnet'):
        ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
        if a==0:
            df1 = pd.read_csv(os.path.join(folder,csv))
            df1_ = df1.drop('fname',axis=1)
            df1_ = np.array(df1_) * ratio
            df1_ = pd.DataFrame(df1_)
            df1 = df1.filter(['fname'])
            df1 = pd.merge(df1,df1_,how='inner',right_index=True,left_index=True)
            a+=1
        else:
            df1_name = pd.read_csv(os.path.join(folder,csv))
            df1_ = df1_name.drop('fname',axis=1)
            df1_ = np.array(df1_) * ratio
            df1_ = pd.DataFrame(df1_)
            df1_name = df1_name.filter(['fname'])
            df1_ = pd.merge(df1_name, df1_,how='inner',right_index=True,left_index=True)
            df1.append(df1_, ignore_index=True)
    elif csv.startswith('mike_cnn2d'):
        ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
        if b==0:
            df2 = pd.read_csv(os.path.join(folder,csv))
            b+=1
        else:
            df2.append(pd.read_csv(os.path.join(folder,csv)), ignore_index=True)     
    elif csv.startswith('mow_cnn2d'):
        ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
        if c==0:
            df3 = pd.read_csv(os.path.join(folder,csv))
            c+=1
        else:
            df3.append(pd.read_csv(os.path.join(folder,csv)), ignore_index=True)     
    
    else:
        print('unknown csv')
        break
#         ratio = acc_df[acc_df['csv_name'] == csv]['acc'].values[0]
#         print(ratio)
#         ratio_all += ratio
#     df = pd.read_csv(os.path.join(folder,csv),header=None)
df = pd.merge(df1,df2,on='fname',how='inner')
df = pd.merge(df,df3,on='fname',how='inner')
# if df.iloc[0,0] == 'fname':
df = df.drop('fname',axis=1)
# df = df.drop(0,axis=1)

df

# if i==0:
#     train_X = df.values*ratio
# else:
#     train_X += df.values*ratio
# train_X.shape

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,31,32,33,34,35,36,37,38,39,40
0,9.997416e-01,1.709653e-07,2.035572e-06,1.864504e-04,3.682731e-08,5.241451e-11,9.130471e-08,8.113211e-07,1.103522e-09,5.521541e-08,...,1.297249e-04,5.407543e-05,5.271563e-05,1.089746e-05,1.471048e-05,4.878185e-07,1.515319e-06,3.664388e-05,1.037959e-04,5.338670e-05
1,3.107701e-05,2.114974e-05,2.176542e-05,1.028734e-04,1.051576e-05,4.915812e-02,1.494288e-04,1.273350e-05,1.334608e-01,3.501518e-05,...,3.566924e-04,6.125082e-04,1.979395e-04,3.311027e-04,1.310831e-03,7.448039e-04,7.436033e-06,5.473242e-03,2.183738e-05,9.476125e-01
2,1.230611e-10,4.660821e-04,2.319601e-07,3.348002e-10,9.990396e-01,2.641246e-08,3.110728e-09,5.081297e-08,1.731302e-09,8.962733e-12,...,1.037115e-06,1.202261e-06,4.162490e-07,1.643107e-05,6.369353e-07,9.487566e-07,5.412159e-08,1.711537e-03,5.121020e-05,6.025197e-07
3,3.933459e-07,1.616190e-04,1.304029e-06,2.192181e-05,2.730575e-06,1.962681e-08,7.201833e-07,9.961210e-01,2.719202e-07,1.110865e-05,...,3.867599e-05,1.045946e-04,4.227337e-04,1.349142e-03,6.061484e-04,4.726653e-05,1.916321e-05,1.637565e-03,7.256485e-03,1.195431e-04
4,2.933400e-08,1.212480e-09,1.211804e-08,1.401436e-08,3.707095e-09,6.104653e-08,3.289114e-06,7.680236e-08,1.604803e-01,3.975342e-02,...,1.938433e-05,1.408415e-05,8.286372e-08,3.652232e-06,6.556502e-05,3.593787e-05,6.004879e-05,5.154248e-05,6.247052e-07,2.686631e-05
5,6.078581e-06,6.528639e-08,1.206319e-07,1.025832e-06,1.135465e-07,2.758994e-09,3.412988e-05,7.041569e-07,7.569481e-05,9.316591e-01,...,1.524554e-04,2.388407e-03,2.184255e-06,1.315312e-05,4.065839e-05,1.805933e-04,3.240673e-05,9.768791e-05,2.233137e-04,3.952798e-04
6,2.911499e-05,1.009813e-03,1.937956e-06,1.078432e-05,2.213438e-05,6.626769e-06,7.185022e-03,2.427714e-04,4.390662e-06,2.694843e-08,...,4.877309e-05,2.733362e-06,4.062961e-06,5.759133e-06,1.018916e-06,5.878059e-05,2.928943e-06,1.277455e-05,2.163622e-05,9.409900e-08
7,5.350362e-09,8.020975e-09,1.470920e-07,1.210292e-08,4.804880e-08,2.018583e-06,3.280496e-06,9.800305e-09,9.866234e-01,1.971264e-05,...,2.236871e-03,5.007052e-05,2.260408e-06,1.227115e-04,3.031397e-04,5.768463e-02,1.365724e-04,5.654457e-04,3.649395e-04,1.778915e-04
8,7.570827e-07,1.214467e-05,2.204143e-08,1.597278e-06,7.301497e-08,2.702426e-09,6.536546e-05,4.519038e-08,3.771726e-07,8.435790e-05,...,6.574093e-08,5.298628e-08,4.938365e-08,1.605109e-08,3.280046e-06,4.043691e-07,1.247291e-07,4.745609e-09,2.919008e-05,3.692595e-08
9,8.189055e-05,3.531236e-06,8.739804e-07,1.820202e-05,3.679770e-06,1.915266e-05,1.191280e-02,5.019204e-06,2.017825e-02,3.000735e-02,...,1.187980e-03,2.147748e-05,1.236939e-07,7.388143e-05,2.021648e-05,9.550547e-03,2.930603e-05,9.217590e-05,1.066246e-06,1.160291e-02


In [39]:
df1 = pd.read_csv(os.path.join(folder,csv))
df1_ = df1.drop('fname',axis=1)
df1_ = np.array(df1_)
pd.DataFrame(df1_)

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,9.597372e-01,1.526626e-03,9.584320e-03,1.242351e-04,5.481070e-06,2.775232e-08,3.137799e-05,5.123215e-06,2.049602e-07,2.429029e-07,...,2.235427e-02,3.897462e-06,9.863401e-05,1.810315e-06,9.795154e-06,3.117769e-07,3.168445e-04,5.767563e-06,7.197126e-05,2.837221e-07
1,2.724478e-03,3.757953e-03,1.256438e-02,1.212118e-03,1.825100e-03,7.329771e-03,7.332546e-05,3.315927e-04,3.480954e-02,8.749823e-04,...,3.001268e-04,2.922729e-02,9.483215e-02,5.252931e-04,1.809503e-01,7.033843e-03,3.223875e-04,8.836229e-03,3.909118e-03,3.879193e-01
2,5.234243e-09,2.136730e-07,3.424003e-09,2.653443e-11,9.999381e-01,1.009137e-10,1.178296e-08,1.655487e-09,1.240144e-08,1.419237e-08,...,2.378844e-07,1.446407e-08,1.186938e-10,5.626341e-09,6.582317e-08,5.778833e-06,2.165666e-10,3.487637e-08,1.881640e-08,2.070535e-12
3,8.745981e-06,9.551656e-01,3.057556e-04,1.797256e-04,7.464732e-03,2.263876e-04,2.771560e-04,5.985004e-03,3.598941e-06,2.903592e-06,...,3.182945e-05,4.087707e-05,2.365388e-04,1.456876e-03,1.755254e-04,2.240974e-04,1.421818e-06,1.506804e-03,1.228464e-02,1.233402e-06
4,1.604206e-06,1.169317e-05,1.073233e-04,1.285149e-06,4.359101e-05,8.377584e-05,4.843712e-05,5.199870e-05,5.378221e-01,3.285487e-01,...,2.222734e-05,5.094003e-05,8.338708e-09,1.411124e-05,6.935948e-05,6.035794e-03,4.368303e-05,1.492470e-05,4.411705e-07,7.967836e-06
5,5.913112e-08,2.232557e-06,8.329609e-07,1.096979e-07,4.649094e-05,1.753039e-05,1.371827e-04,8.082628e-06,6.481736e-04,7.070550e-01,...,1.062394e-04,1.297167e-04,3.843675e-10,2.339748e-05,3.189283e-03,1.006140e-03,1.376888e-05,6.509689e-07,9.979129e-07,7.048799e-06
6,1.947230e-04,1.061385e-03,3.868673e-05,9.120217e-05,8.693869e-05,1.003149e-04,4.224488e-01,1.864442e-04,2.765294e-04,1.046462e-05,...,3.468250e-03,1.288418e-04,8.385631e-04,5.645786e-04,9.301081e-05,6.706365e-04,1.448476e-04,6.190375e-03,8.123507e-04,2.134622e-05
7,3.736596e-08,1.717677e-07,5.081529e-06,1.034508e-06,1.028104e-05,1.245535e-05,6.302907e-06,1.219646e-06,9.507705e-01,5.188566e-04,...,2.513952e-06,5.958019e-07,4.265207e-10,1.669227e-06,3.855648e-05,5.909505e-05,4.641779e-05,2.435083e-05,4.925473e-08,4.221972e-05
8,4.262042e-10,1.827039e-05,7.240915e-09,2.594980e-07,6.218652e-08,7.202672e-07,5.113707e-07,1.713824e-08,1.505726e-08,2.430482e-06,...,1.084775e-06,1.319122e-06,2.296106e-09,2.087446e-08,2.452989e-05,1.797836e-06,1.727613e-07,3.881701e-09,7.121209e-07,1.193100e-07
9,5.770789e-07,2.459657e-05,2.844706e-06,1.912491e-06,1.066850e-06,3.485044e-04,9.037310e-03,3.278851e-04,2.874092e-02,3.309395e-03,...,2.911998e-04,1.704143e-05,7.875188e-09,2.822554e-06,1.627439e-03,5.420589e-02,9.201463e-05,2.422401e-05,4.319050e-06,2.933259e-04


## Split Data to eval

In [7]:
def split_valid_set(X_all, Y_all, percentage):
    all_data_size = len(X_all)
    valid_data_size = int(floor(all_data_size * percentage))

    X_all, Y_all = _shuffle(X_all, Y_all)

    X_train, Y_train = X_all[0:valid_data_size], Y_all[0:valid_data_size]
    X_valid, Y_valid = X_all[valid_data_size:], Y_all[valid_data_size:]

    return X_train, Y_train, X_valid, Y_valid

In [8]:
def _shuffle(X, Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], Y[randomize])

In [9]:
X_train, Y_train, X_valid, Y_valid = split_valid_set(train_X, label, 0.9)
print(X_train.shape , Y_train.shape)

IndexError: index 2589 is out of bounds for axis 1 with size 1895